## hold-out

In [24]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold, RepeatedKFold
from sklearn.pipeline import Pipeline

In [2]:
df = sns.load_dataset('tips')
y_col = 'tip'
X = df.drop(columns=[y_col])
numeric_cols = X.select_dtypes(include=np.number).columns.to_list()
X = pd.get_dummies(X, drop_first=True)
y = df[y_col]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [3]:
scaler = StandardScaler()
X_train_scaled = X_train.copy()
X_train_scaled[numeric_cols] = scaler.fit_transform(X_train[numeric_cols])
X_test_scaled = X_test.copy()
X_test_scaled[numeric_cols] = scaler.transform(X_test[numeric_cols])

In [4]:
model = LinearRegression()
model.fit(X_train_scaled, y_train)
y_pred = model.predict(X_test_scaled)

In [5]:
mean_squared_error(y_test, y_pred)

0.955080898861715

## LOOCV

In [6]:
# X = df['total_bill'].values.reshape(-1, 1)
X = df[['total_bill']]
y = df['tip']

In [10]:
loo = LeaveOneOut()

In [12]:
model = LinearRegression()
mse_list = []
for train_index, test_index in loo.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    mse_list.append(mse)

In [15]:
print(f"MSE(LOOCV):{np.mean(mse_list)}")
print(f"std:{np.std(mse_list)}")

MSE(LOOCV):1.0675673489857438
std:2.0997944551776313


In [18]:
cv = LeaveOneOut()
scores = cross_val_score(model, X, y, cv=cv, scoring='neg_mean_squared_error')
print(f"MSE(LOOCV):{np.mean(scores)}")
print(f"std:{np.std(scores)}")

MSE(LOOCV):-1.0675673489857438
std:2.0997944551776313


## k-Fold CV

In [22]:
k = 5
n_repeats = 3

cv = RepeatedKFold(n_splits=k, n_repeats=3, random_state=0)
model = LinearRegression()
scores = cross_val_score(model, X, y, cv=cv, scoring='neg_mean_squared_error')
print(f"MSE({k}FoldCV): {-np.mean(scores)}")
print(f"std: {np.std(scores)}")


MSE(5FoldCV): 1.0746387233165982
std: 0.2651717854089844


## Pipeline

### Pipeline + KFold

In [26]:
pipeline = Pipeline(steps=[('scaler', StandardScaler()), ('model', LinearRegression())])
cv = KFold(n_splits=5, shuffle=True, random_state=0)
scores = cross_val_score(pipeline, X, y, scoring='neg_mean_squared_error', cv=cv)
scores

array([-0.82130906, -1.07458421, -1.08801239, -1.33238677, -1.084763  ])

In [27]:
pipeline

Pipeline(steps=[('scaler', StandardScaler()), ('model', LinearRegression())])